## More Circuit Identities

In [2]:
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.visualization import array_to_latex
from qiskit.circuit import Gate
from math import pi
c = 0 # control qubit
t = 1 # target  qubit

```
We might not always be able to implement more complicated gates from scratch
which is why we use simpler gates and their identities to construct these gates
```
---

```
Making Controlled Z from CNOT
HXH = Z

Therefore,

(I (*) H) CNOT (I (*) H) = CZ
```

In [4]:
qc = QuantumCircuit(2)
qc.h(t)
qc.cx(c, t)
qc.h(t)
qc.draw()

q_0: ───────■───────
     ┌───┐┌─┴─┐┌───┐
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘

In [5]:
# The same could be done on CNOT to get any rotation by pi
# Controlled Y using CNOT
qc = QuantumCircuit(2)
qc.sdg(t)
qc.cx(c,t)
qc.s(t)
qc.draw()

q_0: ─────────■───────
     ┌─────┐┌─┴─┐┌───┐
q_1: ┤ Sdg ├┤ X ├┤ S ├
     └─────┘└───┘└───┘

In [16]:
# Controlled H using CNOT
# Verifying the above with qiskit
qc = QuantumCircuit(2)
qc.ry(pi/4,t)
qc.cx(c,t)
qc.ry(-pi/4,t)
display(qc.draw()) 

usim = Aer.get_backend('aer_simulator')
qc.save_unitary()

qobj = assemble(qc)
unitary = usim.run(qobj).result().get_unitary()
display(array_to_latex(unitary, prefix="\\text{Circuit = }\n"))

q_0: ─────────────■──────────────
     ┌─────────┐┌─┴─┐┌──────────┐
q_1: ┤ Ry(π/4) ├┤ X ├┤ Ry(-π/4) ├
     └─────────┘└───┘└──────────┘

<IPython.core.display.Latex object>

In [18]:
# Verifying the above with qiskit
qc = QuantumCircuit(2)
qc.swap(0,1)
display(qc.draw()) 

qc.save_unitary()
usim = Aer.get_backend('aer_simulator')
qobj = assemble(qc)
unitary = usim.run(qobj).result().get_unitary()
display(array_to_latex(unitary, prefix="\\text{Circuit = }\n"))

qc = QuantumCircuit(2)
qc.cx(0,1)
qc.cx(1,0)
qc.cx(0,1)
display(qc.draw())
qc.save_unitary()

qobj = assemble(qc)
unitary = usim.run(qobj).result().get_unitary()
array_to_latex(unitary, prefix="\\text{Circuit = }\n")

q_0: ─X─
      │ 
q_1: ─X─

<IPython.core.display.Latex object>

┌───┐     
q_0: ──■──┤ X ├──■──
     ┌─┴─┐└─┬─┘┌─┴─┐
q_1: ┤ X ├──■──┤ X ├
     └───┘     └───┘

<IPython.core.display.Latex object>

```
This works for states |00>, |01>, |10>, |11> and if it works
on for all states in the computational basis, then it must work
for all states generally
```

In [27]:
# Controlled Rotation around the y axis
theta = pi # can be any arbitrary angle

qc = QuantumCircuit(2)
qc.ry(theta/2, t)
qc.cx(c,t)
qc.ry(-theta/2, t)
qc.cx(c, t)
display(qc.draw()) 

qc.save_unitary()
usim = Aer.get_backend('aer_simulator')
qobj = assemble(qc)
unitary = usim.run(qobj).result().get_unitary()
display(array_to_latex(unitary, prefix="\\text{Circuit = }\n"))

q_0: ─────────────■────────────────■──
     ┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐
q_1: ┤ Ry(π/2) ├┤ X ├┤ Ry(-π/2) ├┤ X ├
     └─────────┘└───┘└──────────┘└───┘

<IPython.core.display.Latex object>

```
If the control qubit is |0>, then Ry(theta/2) and then Ry(-theta/2) get applied, 
which doesn't change the target qubit
If the control qubit is |1>, then the qubit rotates by theta/2 about y, and then flipped
about x by pi, then rotated by -theta/2 about y, and then flipped about x by pi again
(use a pencil and perform the rotations)
This is the same as rotating about Ry by theta/2 and then making that same rotation again
(and thus achieving a net rotation about y by theta)

The X gates effectively flip the direction of rotation (possible because x and y are orthogonal)

We can do something similar for Controlled-Rz and Controlled-Rx as well
```

In [29]:
# For a general single qubit rotation V, find 3 rotations, A, B, and C and a phase alpha, such that
# V = e^(i*alpha) AZBZC
# Then use controlled-Z for when the control qubit is |0>, and controlled-Z again for when the control qubit is |1>

A = Gate('A', 1, [])
B = Gate('B', 1, [])
C = Gate('C', 1, [])
alpha = 1

qc = QuantumCircuit(2)
qc.append(C, [t])
qc.cz(c,t)
qc.append(B, [t])
qc.cz(c,t)
qc.append(A, [t])
qc.p(alpha, c)
qc.draw()

┌──────┐
q_0: ──────■───────■─┤ P(1) ├
     ┌───┐ │ ┌───┐ │ └┬───┬─┘
q_1: ┤ C ├─■─┤ B ├─■──┤ A ├──
     └───┘   └───┘    └───┘

In [40]:
# Toffoli Gate / Controlled-Controlled NOT Gate
# Performs X on the target qubit only if both the control qubits are |1>

qc = QuantumCircuit(3)
a = 0
b = 1
target = 2
qc.ccx(a,b,target)

display(qc.draw())
usim = Aer.get_backend('aer_simulator')
qc.save_unitary()

qobj = assemble(qc)
unitary = usim.run(qobj).result().get_unitary()
display(array_to_latex(unitary, prefix="\\text{Circuit = }\n"))

q_0: ──■──
       │  
q_1: ──■──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘

<IPython.core.display.Latex object>

In [34]:
# Controlled-Controlled-U Gate for any arbitrary single qubit rotation
# We need to obtain V = root(U), and then obtain V_dag
# Then obtain Controlled-V and Controlled-V_dag, and using these 2, create the Controlled-Controlled-U

# Using the controlled-phase gate as controlled-V for now
qc = QuantumCircuit(3)
a = 0
b = 1
target = 2

qc.cp(theta, b, target)
qc.cx(a,b)
qc.cp(-theta, b, target)
qc.cx(a,b)
qc.cp(theta, a, target)
qc.draw()

q_0: ─────────■────────────■───■─────
            ┌─┴─┐        ┌─┴─┐ │     
q_1: ─■─────┤ X ├─■──────┤ X ├─┼─────
      │P(π) └───┘ │P(-π) └───┘ │P(π) 
q_2: ─■───────────■────────────■─────

```
The U gate is applied if and only if both the control qubits are 1
```

In [38]:
# Controlled-Controlled-X Gate with H, CX, T, T_dag, SWAP

qc = QuantumCircuit(3)
a = 0
b = 1
target = 2

qc.h(target)
qc.cx(b, target)
qc.tdg(target)
qc.cx(a,target)
qc.t(target)
qc.cx(b,target)
qc.tdg(target)
qc.cx(a,target)
qc.barrier()
qc.t(b)
qc.t(target)
qc.h(target)
qc.swap(b,target)
qc.cx(a,target)
qc.t(a)
qc.tdg(target)
qc.cx(a,target)
qc.swap(b,target)

display(qc.draw())
usim = Aer.get_backend('aer_simulator')
qc.save_unitary()

qobj = assemble(qc)
unitary = usim.run(qobj).result().get_unitary()
display(array_to_latex(unitary, prefix="\\text{Circuit = }\n"))

░                    ┌───┐ »
q_0: ───────────────────■─────────────────────■───░────────────────■───┤ T ├─»
                        │                     │   ░ ┌───┐          │   └───┘ »
q_1: ───────■───────────┼─────────■───────────┼───░─┤ T ├──────X───┼─────────»
     ┌───┐┌─┴─┐┌─────┐┌─┴─┐┌───┐┌─┴─┐┌─────┐┌─┴─┐ ░ ├───┤┌───┐ │ ┌─┴─┐┌─────┐»
q_2: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├─░─┤ T ├┤ H ├─X─┤ X ├┤ Tdg ├»
     └───┘└───┘└─────┘└───┘└───┘└───┘└─────┘└───┘ ░ └───┘└───┘   └───┘└─────┘»
«             
«q_0: ──■─────
«       │     
«q_1: ──┼───X─
«     ┌─┴─┐ │ 
«q_2: ┤ X ├─X─
«     └───┘

<IPython.core.display.Latex object>

In [41]:
# Another way of implementing AND with quantum gates
# (introduces some relative phases)
qc = QuantumCircuit(3)
a = 0
b = 1
target = 2

qc.ch(a,target)
qc.cz(b,target)
qc.ch(a,target)
qc.draw()

q_0: ──■───────■──
       │       │  
q_1: ──┼───■───┼──
     ┌─┴─┐ │ ┌─┴─┐
q_2: ┤ H ├─■─┤ H ├
     └───┘   └───┘

```
All q_0, q_1, q_2 can take only values 0 or 1
For |q_0 q_1> (little Endian Notation = Qiskit Notation)
|00> does not get affected                                                 
[therefore, no observable effect]

|01> applies only Z, which introduces a relative phase in the target qubit
[therefore, no observable effect]

|10> applies H, and then H again, which doesn't change the target qubit
[therefore, no observable effect]

|11> applies HZH = X, which flips the target qubit
[target: |0> becomes |1>, |1> becomes |0>]
```

---
```
It's not possible to implement single qubit rotations by angle theta precisely
"There will always be a limit to the accuracy we can achieve, and it will always be larger than is tolerable when we account for the build-up of imperfections over large circuits."

In fault tolerant systems, we use H and T
(We can perfect H and T using error correction)

T is a rotation around the z-axis by pi/4
Therefore, T = Rz(pi/4) = e^(i * pi/8 * Z)

```

In [42]:
# Making Rx(pi/4)
qc = QuantumCircuit(1)
qc.h(0)
qc.t(0)
qc.h(0)
qc.draw()

┌───┐┌───┐┌───┐
q: ┤ H ├┤ T ├┤ H ├
   └───┘└───┘└───┘

In [43]:
# Gate Rz(pi/4).Rx(pi/4)
# Apply Rx first, and then Rz
qc = QuantumCircuit(1)
qc.h(0)
qc.t(0)
qc.h(0)
qc.t(0)
qc.draw()

┌───┐┌───┐┌───┐┌───┐
q: ┤ H ├┤ T ├┤ H ├┤ T ├
   └───┘└───┘└───┘└───┘

```
Range of angle is [0, 2*pi]
Split this into n slices of width 2*pi/n
For n+1 rotations, at least one slice will contain 2 angles (theta1, theta2) (by the pigeonhole principle)
n1 = number of repetitions required for theta1
n2 = number of repetitions required for theta2

Doing n2 - n1 repetitions is the same as rotating by (theta2) and then by (-theta1)
Angle produced by n2-n1 repetitions is theta(n2-n1)

theta(n2-n1) != 0 and,
-2*pi/n <= theta(n2-n1) <= 2*pi/n

(Rotation by small angles)
```

In [3]:
# For rotation around another axis,
# Gate Rx(pi/4).Rz(pi/4)
qc = QuantumCircuit(1)
qc.t(0)
qc.h(0)
qc.t(0)
qc.h(0)
qc.draw()

┌───┐┌───┐┌───┐┌───┐
q: ┤ T ├┤ H ├┤ T ├┤ H ├
   └───┘└───┘└───┘└───┘

```
This axis is different from the one when Rx is applied first, and then Rz is applied
With these 2 axes, we can perform an arbitrary rotation in the bloch sphere
Since T gates are prominent, the complexity of algorithms for fault tolerant quantum computers
is quoted in terms of how many T gates they need
```